In [1]:
import time
import emoji
import re
import random
import requests
import pandas as pd
from bs4 import BeautifulSoup

def random_headers():
    headers_list = [{'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0'},{'User-agent':' Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'},{'User-agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.106 Safari/537.36 OPR/38.0.2220.41'},{'User-agent':'Mozilla/5.0 (Linux; U; Android 4.0.3; de-ch; HTC Sensation Build/IML74K) AppleWebKit/534.30 (KHTML, like Gecko) Version/4.0 Mobile Safari/534.30'},{'User-agent':'Mozilla/5.0 (compatible; MSIE 9.0; Windows Phone OS 7.5; Trident/5.0; IEMobile/9.0)'}]
    headers=random.choice(headers_list)
    #print(headers)
    return headers
    

"""  
def get_main_body(url,headers):
   
    ss=requests.session()
    res=ss.get(url,headers=headers)
    cookies=ss.cookies.get_dict()
    #print(ss.cookies.get_dict())

    res=ss.get(url,headers=headers,cookies=cookies)
    soup=BeautifulSoup(res.text,'html.parser')
    return soup

"""  

def preprocess (x):
    x = emoji.get_emoji_regexp().sub(r'', x)
    x = re.sub(r'[［(（【{][^】)}）］]*[】)}）］]', '', x)
    x = x.strip().replace(" ", "")
    x = re.sub(r'[~\-。]', "", x) 
    return x

headers = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'}
domain = 'https://icook.tw/'
url = 'categories/206'

c = 0
full_url = domain + url
df = pd.DataFrame(columns = ['Recipe Name','Author','Recipe URL','Recipe Picture','Servings','Time','Ingredients',\
                             'Unit','Recipe Like','Publish Date','Views Number','Recipe Steps'])
res = requests.get(full_url, headers=headers)
soup = BeautifulSoup(res.text,'html.parser')
vip_count=0
page = 1

time_start = time.time()



total_amount = soup.select('title')[0].text
total_amount = int(''.join(re.findall("\d+", total_amount)))
max_page = 0
if total_amount % 18 == 0:
    max_page = total_amount // 18
else:
    max_page = (total_amount // 18) + 1



while(page <= max_page):
    page_index = f'?page={page}'
    full_url = domain + url + page_index
    print(f"URL: {full_url}")
    random_headers()
    headers=headers
    res = requests.get(full_url, headers=headers)
    soup = BeautifulSoup(res.text,'html.parser')
    food_list = soup.select('a.browse-recipe-touch-link')
    for food in food_list:
        recipe_url = 'https://icook.tw/' + food['href']
         
        res_recipe = requests.get(recipe_url,headers=headers)
        soup_recipe = BeautifulSoup(res_recipe.text,'html.parser')
        
        print(recipe_url)
        #recipe_title = "".join((soup_recipe.select('h1.title')[0].text).split())
        #recipe_title = preprocess(recipe_title)
        recipe_title = soup_recipe.select('h1.title')[0].text
        recipe_title = preprocess(recipe_title)
        check_vip=soup_recipe.select('h1.title')[0].select('span.badge-vip')
        if len(check_vip)<=0:
            pass
            #print("is not vip data, data will be save")
        else:
            print("this is vip data,data will not save")
            vip_count+=1
            continue
        recipe_author = soup_recipe.select('a.author-name-link')[0].text
        recipe_pic_url = soup_recipe.select("img.main-pic")[0]['src']
        
    
        if len(soup_recipe.select('span.stat-content')) == 2:
            recipe_like = 0
        else:
            if '萬' in soup_recipe.select('span.stat-content')[0].text:
                recipe_like = int(soup_recipe.select('span.stat-content')[0].text[:-5].replace('.','')+"000")
            else:
                recipe_like = int(soup_recipe.select('span.stat-content')[0].text[:-3].replace(',',''))
        
        if not soup_recipe.select('div.servings-info.info-block') and not soup_recipe.select('div.time-info.info-block'):
            recipe_servings = 0
            recipe_time = 0
        elif not soup_recipe.select('div.time-info.info-block'):
            recipe_servings = int(soup_recipe.select('span.num')[0].text)
            recipe_time = 0
        elif  not soup_recipe.select('div.servings-info.info-block'): 
            recipe_servings = 0
            recipe_time = int(soup_recipe.select('span.num')[0].text)
        else: 
            recipe_servings = int(soup_recipe.select('span.num')[0].text)
            recipe_time = int(soup_recipe.select('span.num')[1].text)

        recipe_ingredient = ""
        for ingredient in soup_recipe.select('.ingredient-search'):
            if ingredient.text == "請參考圖片說明":
                continue
            else:
                recipe_ingredient += ingredient.text + " " 
        recipe_ingredient = emoji.get_emoji_regexp().sub(r'', recipe_ingredient)
        
        recipe_unit = ""
        for unit in soup_recipe.select('div.ingredient-unit'):
            if unit.text == "内文有":
                continue
            else:
                recipe_unit += unit.text + " "

        recipe_steps = ""
        counter = 1
        for steps in soup_recipe.select('div.step-instruction-content'):
            recipe_steps += str(counter) + " " + steps.text + "\n"
            recipe_steps = emoji.get_emoji_regexp().sub(r'', recipe_steps)
            counter += 1
            
        publish_date = soup_recipe.select('span.meta-content')
        if len (publish_date)<=0:
            print("publish_date is na")
            publish_date="na"
        else:
            publish_date = publish_date[0].text[:-3]
        
        views = soup_recipe.select('span.meta-content')
        if len(views)<=0:
            print("view is na")
            views_number=0
            views="0"
        else:
            views=views[1].text[:-3]
            if '萬' in views:
                views_number = views[:-2]
                views_number = int(views_number.replace('.','')+"000")
            else:
                views_number = int(views.replace(',',''))
        
        
        
        #print(recipe_section)    
        #print(recipe_url)
        #print(recipe_title)
        #print(recipe_author)
        #print(recipe_pic_url)
        
        print(recipe_like)
        #print(recipe_servings)
        #print(recipe_time)
        #print(recipe_ingredient)
        #print(recipe_unit)
        #print(recipe_steps)
        print(publish_date)
        #print(views)
        print("----")
        
        ls = [recipe_title,recipe_author,recipe_url,recipe_pic_url,recipe_servings,recipe_time,recipe_ingredient,\
              recipe_unit,recipe_like,publish_date,views_number,recipe_steps]
        df.loc[c] = ls
        c += 1
    
        time.sleep(random.randint(1,4))
    page += 1        
df
print('有多少vip資料{}'.format(vip_count))
time_end = time.time()
print(time_end - time_start)


URL: https://icook.tw/categories/206?page=1
https://icook.tw//recipes/287702
1861
2019/03/06
----
https://icook.tw//recipes/238037
2875
2018/01/25
----
https://icook.tw//recipes/339615
60
2020/06/17
----
https://icook.tw//recipes/339711
7
2020/06/14
----
https://icook.tw//recipes/339360
44
2020/06/11
----
https://icook.tw//recipes/339907
9
2020/06/07
----
https://icook.tw//recipes/339799
7
2020/06/06
----
https://icook.tw//recipes/339824
26
2020/06/06
----
https://icook.tw//recipes/339816
16
2020/06/06
----
https://icook.tw//recipes/339664
8
2020/06/05
----
https://icook.tw//recipes/339601
4
2020/06/05
----
https://icook.tw//recipes/339645
7
2020/06/04
----
https://icook.tw//recipes/339471
21
2020/06/04
----
https://icook.tw//recipes/339535
14
2020/06/04
----
https://icook.tw//recipes/339600
26
2020/06/04
----
https://icook.tw//recipes/339316
5
2020/06/04
----
https://icook.tw//recipes/339578
55
2020/06/04
----
https://icook.tw//recipes/339391
6
2020/06/03
----
URL: https://icook.tw/ca

https://icook.tw//recipes/330161
15
2020/04/04
----
https://icook.tw//recipes/329620
29
2020/04/04
----
https://icook.tw//recipes/330050
12
2020/04/03
----
https://icook.tw//recipes/330039
21
2020/04/03
----
https://icook.tw//recipes/329998
6
2020/04/03
----
https://icook.tw//recipes/330008
12
2020/04/03
----
https://icook.tw//recipes/329970
24
2020/04/03
----
https://icook.tw//recipes/329835
56
2020/04/02
----
https://icook.tw//recipes/329820
10
2020/04/02
----
https://icook.tw//recipes/329724
8
2020/04/01
----
https://icook.tw//recipes/329730
4
2020/04/01
----
https://icook.tw//recipes/329692
11
2020/04/01
----
URL: https://icook.tw/categories/206?page=10
https://icook.tw//recipes/329634
4
2020/04/01
----
https://icook.tw//recipes/329661
13
2020/04/01
----
https://icook.tw//recipes/329621
131
2020/04/01
----
https://icook.tw//recipes/329605
379
2020/04/01
----
https://icook.tw//recipes/329592
12
2020/03/31
----
https://icook.tw//recipes/329588
268
2020/03/31
----
https://icook.tw//re

https://icook.tw//recipes/323958
32
2020/02/18
----
https://icook.tw//recipes/323987
124
2020/02/18
----
https://icook.tw//recipes/323887
16
2020/02/17
----
https://icook.tw//recipes/323816
18
2020/02/17
----
https://icook.tw//recipes/323801
305
2020/02/17
----
URL: https://icook.tw/categories/206?page=18
https://icook.tw//recipes/323699
96
2020/02/16
----
https://icook.tw//recipes/323646
47
2020/02/16
----
https://icook.tw//recipes/323641
89
2020/02/15
----
https://icook.tw//recipes/323601
31
2020/02/15
----
https://icook.tw//recipes/323582
14
2020/02/15
----
https://icook.tw//recipes/323487
34
2020/02/14
----
https://icook.tw//recipes/323504
133
2020/02/14
----
https://icook.tw//recipes/323466
75
2020/02/14
----
https://icook.tw//recipes/323448
37
2020/02/14
----
https://icook.tw//recipes/323376
668
2020/02/13
----
https://icook.tw//recipes/323372


IndexError: list index out of range

In [19]:
df

,Recipe Name,Author,Recipe URL,Recipe Picture,Servings,Time,Ingredients,Unit,Recipe Like,Publish Date,Views Number,Recipe Steps
0,抹茶奶酥抹醬,hey_roommate,https://icook.tw//recipes/340057,https://imageproxy.icook.network/resize?height...,4,20,糖粉 奶粉 抹茶粉 奶油,30g 30g 20g 70g,17,2020/06/08,468,1 奶油退冰後拌開\n\n歡迎追蹤IG：hey_roommate\n2 加入糖粉、奶粉、抹茶...
1,自製抹茶奶酥抹醬,Kylie ✈️,https://icook.tw//recipes/339911,https://imageproxy.icook.network/resize?height...,10,15,無鹽奶油 糖粉 奶粉 抹茶粉,113g 50g 60g 5g,11,2020/06/07,408,1 奶油要先放置室溫軟化，軟化到可以輕輕一磋就陷下去\n2 軟化後用攪拌器將奶油打成乳白色，...
2,清涼消暑鳳梨醬,Natasha,https://icook.tw//recipes/339850,https://imageproxy.icook.network/resize?height...,4,90,鳳梨 冰糖 檸檬,1公斤 300克 1顆,19,2020/06/06,882,1 鳳梨量就用一顆，削完皮後差不多1公斤，糖可以以平常吃得甜度下去調整，我平常飲料吃無糖、微...
3,火龍果果醬️,🍓草莓奶凍🍰,https://icook.tw//recipes/339423,https://imageproxy.icook.network/resize?height...,0,0,火龍果 白砂糖 鹽,約700克 200克 一戳,7,2020/06/03,352,1 （1）火龍果皮有豐富天然果膠和花青素勿 \n 丟，先把外皮有綠葉的地方...
4,生酮版巧克力蛋白抹醬,小翎的生酮飲食實驗室,https://icook.tw//recipes/338887,https://imageproxy.icook.network/resize?height...,0,30,水 羅漢果糖 無糖可可粉 乳清蛋白粉 酪梨油,60ml 6大匙 3大匙 5大匙 2大匙,6,2020/05/29,475,1 本次使用的乳清蛋白粉為奶油餅乾口味，除了水果口味不適合外，其他任何口味均可。\n2 水，...
...,...,...,...,...,...,...,...,...,...,...,...,...
695,百香蘋果醬,Yvonne Kuan,https://icook.tw//recipes/16149,https://imageproxy.icook.network/resize?height...,0,0,百香果剖開去籽 蘋果去皮和籽切細丁 糖,350g 120g 260g,428,2012/01/09,16000,1 將百香果連同籽放入調理機中，採低速打30秒，就可以把果肉和籽分開又不會把籽打碎，有點類似...
696,ㄚ曼達的廚房鳳梨蘋果醬,ㄚ曼達,https://icook.tw//recipes/15798,https://imageproxy.icook.network/resize?height...,0,0,鳳梨 蘋果 糖,500克 1個 200克,422,2011/12/27,17000,1 鳳梨切丁備用\n2 鳳梨心也先切小丁\n3 步驟3\n4 鳳梨心丁加一點鳳梨丁用果汁機打...
697,品靚上菜自製草莓果醬XDD,品靚上菜 PJ Cook,https://icook.tw//recipes/15162,https://imageproxy.icook.network/resize?height...,0,0,新鮮草莓 赤砂糖 乾淨且乾燥的容器(玻璃瓶),不限 適量 數個,134,2011/12/05,8993,1 作法很簡單，只要將洗乾淨的草莓加紅砂糖一起煮就可以了，不用加水唷~\n這次有把泡泡撈掉，...
698,《Peggy廚房》Peggy牌手工蔓越莓桑葚果醬,Peggy Chang,https://icook.tw//recipes/15152,https://imageproxy.icook.network/resize?height...,0,0,新鮮蔓越莓 新鮮桑葚 新鮮檸檬汁 砂糖甜度自訂 吉利丁粉不夠凝固再增加 乾淨玻璃瓶看果醬多寡,450克 320克 半顆 300~350克 0.5大匙 2~4個,813,2011/12/05,29000,1 1‧新鮮蔓越莓和桑葚分別用清水洗乾淨，把水份瀝乾。先把蔓越莓放進果汁機裡打碎，倒入鍋中(...


In [20]:
df.to_csv(f'./{section_number}.csv', index=False ,encoding='utf-8-sig')